In [ ]:
%matplotlib inline

import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import tensorflow.keras
import tensorflow.keras.callbacks
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard

In [ ]:
import pandas as pd
import io

df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
headlines = df.headline.tolist()
text=""
#clean the data into individual sentences

for headline in headlines[:5000]:
    text=text+" "+headline[0:len(headline)]+'.'

# avg_len = sum( map(len, headlines) ) / len(headlines)
# avg_len
text = text[1:]
text[:2000]

In [ ]:
# Adapted from Week 5 Lab

# storing all unique characters

chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


# breaking the data into sentences of length seqlen
seqlen = 100
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen])

# setting up the data to be trained by the LSTM Model

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Model below

model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Using a high diversity

    for diversity in [0.8, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

In [ ]:
headlines = df.headline.tolist()
sentences_2=[]
text2=""

total_avg = sum( map(len, headlines) ) / len(headlines)
avg_len_of_sen=int(total_avg)

for headline in headlines:
    if len(headline) == avg_len_of_sen:
        sentences_2.append(headline[0:len(headline)])
        text2=text2+" "+headline[0:len(headline)]
# text = text[1:]
#text[1:2000]

In [ ]:

chars1 = sorted(list(set(text2)))
char_indices1 = dict((c, i) for i, c in enumerate(chars1))
indices_char1 = dict((i, c) for i, c in enumerate(chars1))


seqlen1 = avg_len_of_sen
step1 = seqlen1

x = np.zeros((len(sentences_2), seqlen1, len(chars1)), dtype=np.bool)
y = np.zeros((len(sentences_2), seqlen1, len(chars1)), dtype=np.bool)
for i, sentence in enumerate(sentences_2):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices1[char_in]] = 1
        y[i, t, char_indices1[char_out]] = 1

model1 = Sequential()
model1.add(LSTM(128, input_shape=(seqlen1, len(chars1)), return_sequences=True))
model1.add(Dense(len(chars1), activation='softmax'))

model1.compile(
    loss='categorical_crossentropy',
    optimizer='adam', #RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end1(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    rand_index = random.randint(0, len(sentences_2) - 1)
    
    # Q5: What does diversity do?
    for diversity in [0.8, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = sentences_2[rand_index]
        #sentence=''
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(avg_len_of_sen*4):
            x_pred = np.zeros((1, seqlen1, len(chars1)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices1[char]] = 1.
            
            preds1 = model1.predict(x_pred, verbose=0)
            next_index = sample(preds1[0, -1], diversity)
            next_char = indices_char1[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end1)

model1.fit(x, y,
          batch_size=60,
          epochs=50,
          callbacks=[print_callback])

In [ ]:
headlines = df.headline.tolist()
sentences3=[]

for headline in headlines:
    sentences3.append(headline[0:len(headline)])

sentences3[:200]
subset_sentences = sentences3[:5000]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os

In [ ]:
tokenizer = Tokenizer()

## tokenization

tokenizer.fit_on_texts(subset_sentences)
total_words = len(tokenizer.word_index) + 1
    
## convert data to sequence of tokens

in_sequences = []
for line in subset_sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        sequence = token_list[:i+1]
        in_sequences.append(sequence)

in_sequences[:10]

In [ ]:
max_sequence_len = max([len(x) for x in in_sequences])
in_sequences = np.array(pad_sequences(in_sequences, maxlen=max_sequence_len, padding='pre'))   
X, y = in_sequences[:,:-1],in_sequences[:,-1]
y = ku.to_categorical(y, num_classes=total_words)

In [ ]:
# RNN LSTM model

model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))
model.add(LSTM(100))

# optional

# model.add(Dropout(0.3))

model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

In [ ]:
model.fit(X, y, epochs=10, verbose=5)

In [ ]:
def generate_headlines(seed_text, headline_len, model, max_sequence_len):
    for _ in range(headline_len):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, i in tokenizer.word_index.items():
            if i == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text

In [ ]:
import requests
from bs4 import BeautifulSoup

trends=[]

URL = "https://twitter-trends.iamrohit.in/united-states"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="copyData")
rows = results.find_all('tr')
for row in rows:
    cols=row.find_all('th')
    cols=[x.text for x in cols]
    if len(cols) > 2:
        trend = cols[1].split('\n')[0]
        if trend[0] == '#':
            trends.append(trend[1:-1])
        else:
            trends.append(trend[:-1])
trends

In [ ]:
fake_news=[]
headline_len = 10
for trend in trends:
    print(generate_headlines(trend, headline_len, model, max_sequence_len).title())
    fake_news.append(generate_headlines(trend, headline_len, model, max_sequence_len).title())